In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "https://www.unb.ca/academics/calendar/undergraduate/current/frederictonprograms/bachelorofscienceinengineering/mechanicalengineering.html"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [4]:
driver.get(url)

In [5]:
page_soup = soup(driver.page_source, 'lxml')

# Mechatronics Option

Collect course link texts (for driver to click on)

In [6]:
start = page_soup.find("div", {"id": "stack"}).find("p", text="Core Courses for Mechatronics Option:")
start

<p><strong>Core Courses for Mechatronics Option:</strong></p>

In [7]:
option_core = start.findNext("p")
option_core

<p>Compared to the standard Mechanical Engineering Program, only one technical elective is required (as opposed to two for the standard program). For the Mechatronics option, <a href="../../frederictoncourses/electricalandcomputerengineering/ece-2711.html">ECE 2711</a> (Electric Circuits) is recommended instead of <a href="../../frederictoncourses/electricalandcomputerengineering/ece-2701.html">ECE 2701</a> (Electric Circuits and Electronics for Non-electricals). Also required are: <a href="../../frederictoncourses/electricalandcomputerengineering/ece-2214.html">ECE 2214</a> (Digital Logic Design), <a href="../../frederictoncourses/electricalandcomputerengineering/ece-2215.html">ECE 2215</a> (Digital Logic Design Lab), ECE 3111 (Electronics I), <a href="../../frederictoncourses/mechanicalengineering/me-4673.html">ME 4673</a> (Introduction to Mechatronics) and <a href="../../frederictoncourses/mechanicalengineering/me-4683.html">ME 4683</a> (Mechatronics Applications). The work for the 

In [8]:
option_electives = option_core.findNext("p").findNext("p")
option_electives

<p><a href="../../frederictoncourses/electricalandcomputerengineering/ece-3221.html">ECE 3221</a><span> </span>Computer Organization<span> </span>(4 ch)<br/><a href="../../frederictoncourses/electricalandcomputerengineering/ece-4323.html">ECE 4323</a><span> </span>Industrial Control Systems<span> </span>(4 ch)<br/><a href="../../frederictoncourses/electricalandcomputerengineering/ece-4333.html">ECE 4333<span> </span></a>Robotics<span> </span>(4 ch)<br/><a href="../../frederictoncourses/electricalandcomputerengineering/ece-4343.html">ECE 4343</a> Haptic Robotics (4 ch)<br/><a href="../../frederictoncourses/electricalandcomputerengineering/ece-4923.html">ECE 4923</a><span> </span>Introduction to Biomedical Engineering<span> </span>(4 ch)<br/><a href="../../frederictoncourses/mechanicalengineering/me-4173.html">ME 4173<span> </span></a>Robot Kinematics<span> </span>(4 ch)<br/><a href="../../frederictoncourses/mechanicalengineering/me-4703.html">ME 4703</a> Mechanical Engineering Measureme

In [9]:
containers = option_core.findAll("a") + option_electives.findAll("a")
link_texts = [container.text.strip() for container in containers if container.text.strip() != 'ECE 2701']
link_texts

['ECE 2711',
 'ECE 2214',
 'ECE 2215',
 'ME 4673',
 'ME 4683',
 'ME 4860',
 'ENGG 4000',
 'TME 4025',
 'ECE 3221',
 'ECE 4323',
 'ECE 4333',
 'ECE 4343',
 'ECE 4923',
 'ME 4173',
 'ME 4703',
 'ME 5183',
 'ME 5643',
 'ME 5653',
 'ME 5713',
 'ME 5953']

In [10]:
len(link_texts)

20

Automation script to scrape all course info

In [11]:
from selenium.common.exceptions import NoSuchElementException

course_codes = []
course_names = []
course_descs = []
counter = 0
    
for link_text in link_texts:
    #go to course page
    try:
        link = driver.find_element_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue

    time.sleep(2)
    link.click()    
    time.sleep(2)
    
    #scrape course info
    page_soup = soup(driver.page_source, 'lxml')
    container = page_soup.find("table", {"class": "ugradcal large-only"})
    course_code = container.find("th", {"abbr": "Course Code"}).text
    course_name = container.find("th", {"abbr": "Course Dscription"}).text
    course_desc = container.find("course_description").text.strip()
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    print("Scraped ", course_codes[-1])
    counter += 1
    
    #go to course list page
    driver.back()
    time.sleep(2)
        
print("Finished scraping {} courses".format(counter))

Scraped  ECE2711
Scraped  ECE2214
Scraped  ECE2215
Scraped  ME4673
Scraped  ME4683
Scraped  ME4860
Scraped  ENGG4000
Scraped  TME4025
Scraped  ECE3221
Scraped  ECE4323
Scraped  ECE4333
Scraped  ECE4343
Scraped  ECE4923
Scraped  ME4173*
Scraped  ME4703*
Scraped  ME5183*
Scraped  ME5643*
Scraped  ME5653*
Scraped  ME5713*
Scraped  ME5953*
Finished scraping 20 courses


In [12]:
course_codes

['ECE2711',
 'ECE2214',
 'ECE2215',
 'ME4673',
 'ME4683',
 'ME4860',
 'ENGG4000',
 'TME4025',
 'ECE3221',
 'ECE4323',
 'ECE4333',
 'ECE4343',
 'ECE4923',
 'ME4173*',
 'ME4703*',
 'ME5183*',
 'ME5643*',
 'ME5653*',
 'ME5713*',
 'ME5953*']

In [13]:
course_names

['Electric Circuits',
 'Digital Logic Design',
 'Digital Logic Design Laboratory ',
 'Introduction to Mechatronics',
 'Mechatronics Applications ',
 'Senior Design Project',
 'Senior Design Project ',
 'Product Design and Development',
 'Computer Organization',
 'Modern Control Systems and Applications ',
 'Robotics',
 'Haptic Robotics',
 'Introduction to Biomedical Engineering',
 'Robot Kinematics',
 'Mechanical Engineering Measurements',
 'Industrial Robotics and Automation',
 'Automatic Controls II ',
 'Predictive Control and Intelligent Sensors',
 'Nondestructive Testing',
 'Embedded Flight Control Systems']

In [14]:
course_descs

['Basic DC circuits: Network analysis and theorems. AC circuits: introduction of phasors, Network analysis and theorems applied to AC circuits.',
 'An introductory course to practical aspects of digital system design. The course covers digital logic design, including basic design concepts and implementation technology, number representation, synthesis of combinational and sequential logic, and the use of HDL and computer-based tools.',
 'This is an introductory course to practical aspects of digital system design. Course includes the design of digital circuits with CAD tools and VHDL hardware description language.',
 'Mechatronics is an integrated approach to mechanical, electronic and computer engineering for the design of “smart” products and “intelligent” manufacturing systems. Fundamentals of mechatronics design, with emphasis on product design and fabrication. Examples of mechanical systems utilizing sensors and actuator technologies, including use of signal conditioning circuits 

In [15]:
course_descs = [desc.replace("\xa0", " ") for desc in course_descs]

In [16]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs
})

df

,Course Number,Course Name,Course Description
0,ECE2711,Electric Circuits,Basic DC circuits: Network analysis and theore...
1,ECE2214,Digital Logic Design,An introductory course to practical aspects of...
2,ECE2215,Digital Logic Design Laboratory,This is an introductory course to practical as...
3,ME4673,Introduction to Mechatronics,Mechatronics is an integrated approach to mech...
4,ME4683,Mechatronics Applications,Concepts in automating processes. System speci...
5,ME4860,Senior Design Project,A mechanical engineering design is developed a...
6,ENGG4000,Senior Design Project,Full-year design course (fall and winter of sa...
7,TME4025,Product Design and Development,Full-year Product Design and Development cours...
8,ECE3221,Computer Organization,"Register transfer systems and datapaths, micro..."
9,ECE4323,Modern Control Systems and Applications,Introduces the industrial context for the appl...


In [17]:
df.to_csv('UNewBrunswick_MechEng_Mechatronics_Option_Core_and_Electives_Courses.csv', index = False)

# Biomedical Option

Collect course link texts (for driver to click on)

In [18]:
page_soup = soup(driver.page_source, 'lxml')

In [19]:
start = page_soup.find("div", {"id": "stack"}).find("p", text="Requirements:")
start

<p><strong>Requirements:</strong></p>

In [20]:
option_core = start.findNext("p")
option_core

<p><a href="../../frederictoncourses/appliedscience/apsc-3953.html">APSC 3953</a>, Basis of Biomedical Engineering (3 ch) <br/><a href="../../frederictoncourses/mechanicalengineering/me-4860.html">ME 4860</a>, or <a href="../../frederictoncourses/engineering/engg-4000.html">ENGG 4000</a> or <a href="../../frederictoncourses/tme/tme-4025.html">TME 4025</a>, Senior project selected to be a biomedical engineering related project. (8 ch)</p>

In [21]:
option_electives1 = option_core.findNext("p").findNext("p")
option_electives1

<p><a href="../../frederictoncourses/mechanicalengineering/me-4173.html">ME 4173</a> Robot Kinematics (4 ch)<br/><a href="../../frederictoncourses/mechanicalengineering/me-4622.html">ME 4622</a> Human Factors Engineering (3 ch)<br/><a href="../../frederictoncourses/mechanicalengineering/me-4703.html">ME 4703</a> Mechanical Measurements (4 ch)<br/><a href="../../frederictoncourses/mechanicalengineering/me-5173.html">ME 5173</a> Advanced Kinematics of Manipulators (3 ch)<br/><a href="../../frederictoncourses/mechanicalengineering/me-5223.html">ME 5223</a> Finite Element Analysis (3 ch)<br/>ME 5622 Human Factors Engineering (3 ch)<br/><a href="../../frederictoncourses/mechanicalengineering/me-5653.html">ME 5653</a>/ME6003 - Predictive Control and Intelligent Sensors (4 ch)<br/><a href="../../frederictoncourses/mechanicalengineering/me-5913.html">ME 5913</a> Biomechanics I (4 ch)<br/>ME 6003 (Special Topics) Fatigue, Failure, and Fracture of Materials (3 ch)<br/>ME 6383 Continuum Mechanics

In [22]:
option_electives2 = option_electives1.findNext("p").findNext("p")
option_electives2

<p><a href="../../frederictoncourses/computer-science/cs-3025.html">CS 3025</a> Human-Computer Interaction (3 ch)<br/><a href="../../frederictoncourses/electricalandcomputerengineering/ece-4343.html">ECE 4343</a> Haptics (4ch)<br/><a href="../../frederictoncourses/kinesiology/kin-3041.html">KIN 3041</a> Disability Awareness (3 ch)<br/><a href="../../frederictoncourses/kinesiology/kin-3061.html">KIN 3061</a> Advanced Biomechanics (4ch)<br/><a href="../../frederictoncourses/kinesiology/kin-3161.html">KIN 3161</a> Human Factors in Ergonomic Design (3ch)<br/><a href="../../frederictoncourses/kinesiology/kin-3041.html">KIN 4041</a> Movement Disorders (3ch)<br/><a href="../../frederictoncourses/kinesiology/kin-4063.html">KIN 4063</a> Biomedical Instrumentation and Data Acquisitions (3ch)<br/><a href="../../frederictoncourses/kinesiology/kin-4161.html">KIN 4161</a> Occupational Biomechanics (3ch)<br/><a href="../../frederictoncourses/kinesiology/kin-4162.html">KIN 4162</a> Occupational Health

In [23]:
containers = option_core.findAll("a") + option_electives1.findAll("a") + option_electives2.findAll("a")
link_texts = [container.text.strip() for container in containers]
link_texts

['APSC 3953',
 'ME 4860',
 'ENGG 4000',
 'TME 4025',
 'ME 4173',
 'ME 4622',
 'ME 4703',
 'ME 5173',
 'ME 5223',
 'ME 5653',
 'ME 5913',
 'CS 3025',
 'ECE 4343',
 'KIN 3041',
 'KIN 3061',
 'KIN 3161',
 'KIN 4041',
 'KIN 4063',
 'KIN 4161',
 'KIN 4162',
 'KIN 4163',
 'PHYS 5993']

In [24]:
len(link_texts)

22

Automation script to scrape all course info

In [25]:
from selenium.common.exceptions import NoSuchElementException

course_codes = []
course_names = []
course_descs = []
counter = 0
    
for link_text in link_texts:
    #go to course page
    try:
        link = driver.find_element_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue

    time.sleep(2)
    link.click()    
    time.sleep(2)
    
    #scrape course info
    page_soup = soup(driver.page_source, 'lxml')
    container = page_soup.find("table", {"class": "ugradcal large-only"})
    course_code = container.find("th", {"abbr": "Course Code"}).text
    course_name = container.find("th", {"abbr": "Course Dscription"}).text
    course_desc = container.find("course_description").text.strip()
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    print("Scraped ", course_codes[-1])
    counter += 1
    
    #go to course list page
    driver.back()
    time.sleep(2)
        
print("Finished scraping {} courses".format(counter))

Scraped  APSC3953
Scraped  ME4860
Scraped  ENGG4000
Scraped  TME4025
Scraped  ME4173*
Scraped  ME4622*
Scraped  ME4703*
Scraped  ME5173*
Scraped  ME5223*
Scraped  ME5653*
Scraped  ME5913*
Scraped  CS3025
Scraped  ECE4343
Scraped  KIN3041
Scraped  KIN3061
Scraped  KIN3161
Scraped  KIN3041
Scraped  KIN4063
Scraped  KIN4161
Scraped  KIN4162
Scraped  KIN4163
Scraped  PHYS5993
Finished scraping 22 courses


In [26]:
course_codes

['APSC3953',
 'ME4860',
 'ENGG4000',
 'TME4025',
 'ME4173*',
 'ME4622*',
 'ME4703*',
 'ME5173*',
 'ME5223*',
 'ME5653*',
 'ME5913*',
 'CS3025',
 'ECE4343',
 'KIN3041',
 'KIN3061',
 'KIN3161',
 'KIN3041',
 'KIN4063',
 'KIN4161',
 'KIN4162',
 'KIN4163',
 'PHYS5993']

In [27]:
course_names

['Basis of Biomedical Engineering',
 'Senior Design Project',
 'Senior Design Project ',
 'Product Design and Development',
 'Robot Kinematics',
 'Human Factors Engineering ',
 'Mechanical Engineering Measurements',
 'Advanced Kinematics of Manipulators',
 'Finite Element Analysis',
 'Predictive Control and Intelligent Sensors',
 'Biomechanics I',
 'Human-Computer Interaction',
 'Haptic Robotics',
 'Disability Awareness',
 'Advanced Biomechanics',
 'Human Factors in Ergonomic Design',
 'Disability Awareness',
 'Biomechanical Instrumentation and Data Acquisition ',
 'Occupational Biomechanics',
 'Occupational Health and Safety for Ergonomists',
 'Workplace Ergonomic Design and Analysis',
 'Magnetic Resonance Imaging (O)']

In [28]:
course_descs

['This course introduces the general aspects of Biomedical Engineering, which is the application of engineering principles to study biology, medicine, behaviour and health. A number of areas are covered by this broad topic, including anatomy, muscle physiology, biomedical instrumentation, biomechanics, and prosthetics. It will also introduce students to the idea of Biomedical Engineering as a career choice.',
 'A mechanical engineering design is developed and documented in the form of a technical report. Students normally work in approved teams. Industrial projects are developed in cooperation with industry and may require some period of time on site. University-based projects are developed in cooperation with university faculty. The first stage of this process involves definition of the project topic, background studies, and development of a conceptual design. An oral examination is conducted towards the end of the first term, and a written preliminary report is submitted. In the seco

In [29]:
course_descs = [desc.replace("\xa0", " ") for desc in course_descs]

In [30]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs
})

df

,Course Number,Course Name,Course Description
0,APSC3953,Basis of Biomedical Engineering,This course introduces the general aspects of ...
1,ME4860,Senior Design Project,A mechanical engineering design is developed a...
2,ENGG4000,Senior Design Project,Full-year design course (fall and winter of sa...
3,TME4025,Product Design and Development,Full-year Product Design and Development cours...
4,ME4173*,Robot Kinematics,Structure and specification of robotic manipul...
5,ME4622*,Human Factors Engineering,An interdisciplinary study of the interaction ...
6,ME4703*,Mechanical Engineering Measurements,Discusses a variety of measurement techniques ...
7,ME5173*,Advanced Kinematics of Manipulators,Various methods for solving the forward and in...
8,ME5223*,Finite Element Analysis,Introduction to the basic concepts of finite e...
9,ME5653*,Predictive Control and Intelligent Sensors,Study on the design and practical implementati...


In [31]:
df.to_csv('UNewBrunswick_MechEng_Biomedical_Option_Core_and_Electives_Courses.csv', index = False)

In [32]:
driver.quit()